# TensorBoard Projector

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

In [ ]:
def visualize_embedding(word_to_vec_map: dict, dir_name: str):
    """
    Creates TensorBoard files to visualize a given word_to_vec_map.

    Arguments:
    word_to_vec_map -- dictionary mapping words to vectors
    dir_name -- name of the directory to be created in TensorBoard logs
    """
    # Create logging directory
    log_dir = os.path.join(os.getcwd(), "logs", dir_name)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Write word-index dictionary
    with open(os.path.join(log_dir, "vectors.tsv"), "w") as vectors, open(os.path.join(log_dir, "metadata.tsv"), "w") as metadata:
        for word, vec in word_to_vec_map.items():
            vectors.write("\t".join([str(x) for x in vec]) + "\n")
            metadata.write(word + "\n")

    with open(os.path.join(log_dir, "vectors.tsv"), "r") as vectors:
        feature_vectors = np.loadtxt(vectors, delimiter="\t")

    # Create checkpoint from embeddings
    weights = tf.Variable(feature_vectors)
    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

    # Configure set-up
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = os.path.join(log_dir, "metadata.tsv")
    projector.visualize_embeddings(log_dir, config)

In [ ]:
%load_ext tensorboard # Select logs dir